In [245]:
#meta: Common data structures in Python - Graphs
#10/28/2018

#exploring two ways of looking at graph relationships:
#  a) focus on edges
#  b) focus on vertices

#10/28/2018
#  b) implemented only uni-directional edges, i.e. one-way flights

## Data Structures - Graphs
src: John Keyser

Graphs offer a structure for capturing an incredibly wide diversity of relationships and 
the connections that are formed all over.  
i.e. connections between locations 

Vertex/node are represented entities.  
Edges - the connections between nodes. Edges let us know that there's a relation between the 2 nodes.    
i.e. cities and roads.  

Codewise, graphs can have more than one representation:
1. global – represent a graph as two lists: one list of nodes and one list of edges 
2. adjacency list - focuses on adjacent neighbors 
3. adjacency matrix 

### 1. Global List
Each node will contain information about itself.  
An edge would reference the two nodes.   
If it's a weighted graph, the edges will also store a weight.  

Most useful if need to regularly look at all of the edges. 

In [195]:
class Node:
    #use __init__() function to assign values
    def __init__(self, name, weight):
        self._name = name
        self._weight = weight
        
    #define getters
    def getName(self):
        return self._name
    def getWeight(self):
        return self._weight
        
class Edge:
    def __init__(self,name1,name2,weight):
        self._name1=name1
        self._name2=name2
        self._weight=weight
        
    #define getters
    def getNodeName1(self):
        return self._name1
    def getNodeName2(self):
        return self._name2
    def getNodeNames(self):
        return (self._name1,self._name2)
    def getDistance(self):
        return self._weight
        

In [196]:
#setup
cities=[]
flights=[]

#create cities - population in 1K units
city = Node('Seattle',705)
cities.append(city)
city = Node('New York',8538)
cities.append(city)
city = Node('Amsterdam',593)
cities.append(city)
city = Node('Kiev',2804)
cities.append(city)
city = Node('Kharkiv',1431)
cities.append(city)
#print(cities[0].getName())

#create connections - air miles
flight = Edge('Seattle', 'New York',  2402.63)
flights.append(flight)
flight = Edge('Seattle', 'Amsterdam',  4863.65 )
flights.append(flight)
flight = Edge('New York', 'Amsterdam',  3598.83 )
flights.append(flight)
flight = Edge('Amsterdam','Kiev', 1107.00 )
flights.append(flight)
flight = Edge('Kiev','Kharkiv',  255.40 )
flights.append(flight)

#### Using edges - i.e. find the total population that lives on a connection
Need to find population of the cities at each end (population is stored in the city nodes.  Connection only knows the names of the cities.

In [197]:
#set the connections ur interested in
flight = flights[0]
city1=flight.getNodeName1()
city2=flight.getNodeName2()

#performance issue: linear => not efficient
#loop thru the list of cities
for i in cities:
    if i.getName()==city1:
        pop1 = i.getWeight()
    if i.getName()==city2:
        pop2 = i.getWeight()

#compute total population
total_pop = pop1 + pop2
print ('Population on connections between {} and {} is {}'.format(city1, city2, total_pop))

Population on connections between Seattle and New York is 9243


Issue: Linear search - loop thru names of the cities.  
If have a large list of cities, not effiecient because is O(n) operation.  
Better: instead of using names of cities, use index of cities. 

In [198]:
#redefine class Edge, use index instead of name => more efficient
class Edge:
    def __init__(self,node1_idx,node2_idx,weight):
        self._node1_idx=node1_idx
        self._node2_idx=node2_idx
        self._weight=weight
        
    #define getters
    def getNode1idx(self):
        return self._node1_idx
    def getNode2idx(self):
        return self._node2_idx
    def getNodeIndices(self):
        return (self._node1_idx, self._node2_idx)
    def getWeight(self):
        return self._weight

In [199]:
#setup
cities=[]
flights=[]

#create cities - population in 1K units
city = Node('Seattle',705)
cities.append(city)
city = Node('New York',8538)
cities.append(city)
city = Node('Amsterdam',593)
cities.append(city)
city = Node('Kiev',2804)
cities.append(city)
city = Node('Kharkiv',1431)
cities.append(city)
#print(cities[0].getName())

#create connections - flights in air miles
flight = Edge(0, 1,  2402.63)
flights.append(flight)
flight = Edge(0, 2,  4863.65 )
flights.append(flight)
flight = Edge(1, 2,  3598.83 )
flights.append(flight)
flight = Edge(2,3, 1107.00 )
flights.append(flight)
flight = Edge(3,4,  255.40 )
flights.append(flight)

In [200]:
#set the connections ur interested in
flight= flights[4]

citi1_idx = flight.getNode1idx()
citi2_idx = flight.getNode2idx()

#compute total population
total_pop = cities[citi1_idx].getWeight() + cities[citi2_idx].getWeight()
print ('Population on connections between {} and {} is {}'.format(cities[citi1_idx].getName(), cities[citi2_idx].getName(), total_pop))


Population on connections between Kiev and Kharkiv is 4235


Using index makes it easier to find the cities, we can jump right to the cities.  
This is a constant time operation O(1) - takes the same time no matter how many cities in the list.  
Downside: the list of cities must remain unchanged.  
Restriction: need to know the index of each city in the list when we set up the edge.  

Indirection - using an index to refer to an item instead of the items' name or value.  Very common, i.e. a dictionary.
#### Bottom line: Global list is optimized for looking at edges

### 2. Adjacency List

Keep a list of edges in each node. 

Is useful in most typical graph operations, I.e. airline connections, social networks, etc. 

Works well because most graph algorithms are already designed to work just looking at one node at a time and its neighbors.  


Let's say want to take a city and find all the connections of that city.  
A painful operation in global approach - have to go thru the list of edges trying to find the edges that have our city as either first or second city in the edge.  

Instead of keeping a global list of all edges in the graph, let's keep a local list of the edges for each node. Each node will keep track of all the edges connected to it.  In this case, it's just one list.  

In [207]:
class Node:
    def __init__(self, name, weight):
        self._name = name
        self._weight = weight
        self._edges = []
        
    #define getters
    def getName(self):
        return self._name
    def getWeight(self):
        return self._weight
    def getEdges(self):
        return self._edges
    
    #define setters
    def addNeighbor(self, neighbor_name, weight=0):
        edge = Edge(neighbor_name, weight)
        self._edges.append(edge)
        
class Edge:
    def __init__(self,neighbor_name,weight=0):
        self._neighbor_name=neighbor_name
        self._weight=weight
        
    #define getters
    def getNeigborName(self):
        return self._neighbor_name
    def getWeight(self):
        return self._weight

In [214]:
#setup
cities=[]

#create nodes: cities with population in 1K units
city = Node('Seattle',705)
cities.append(city)
city = Node('New York',8538)
cities.append(city)
city = Node('Amsterdam',593)
cities.append(city)
city = Node('Kiev',2804)
cities.append(city)
city = Node('Kharkiv',1431)
cities.append(city)
#print(cities[0].getName())


#create edges: connections with air miles
cities[0].addNeighbor(cities[1].getName(),2402.63)
cities[0].addNeighbor(cities[2].getName(),4863.65)
cities[1].addNeighbor(cities[2].getName(),3598.83)
cities[2].addNeighbor(cities[3].getName(),1107.00)
cities[3].addNeighbor(cities[4].getName(),255.40)

#set the city ur interested and see all of its connections
#only uni-directional
for c in cities:
    print('')
    #print(c.getWeight(), "million people")
    
    flights = c.getEdges()
    if len(flights) > 0: 
        for flight in flights:
            print('from ', c.getName(), 'to ', flight.getNeigborName())
            print('flight distance (air miles): ', flight.getWeight())
    else:
        print ('from ', c.getName(),': no flights')



from  Seattle to  New York
flight distance (air miles):  2402.63
from  Seattle to  Amsterdam
flight distance (air miles):  4863.65

from  New York to  Amsterdam
flight distance (air miles):  3598.83

from  Amsterdam to  Kiev
flight distance (air miles):  1107.0

from  Kiev to  Kharkiv
flight distance (air miles):  255.4

from  Kharkiv : no flights


So far, works well but only uni-directional.  To make it bi-directional, need to add an edge to each node every time a flight is created.

### 3. Adjacency matrix 

Instead of list, there are matrix entities that note which nodes are connected.   

Useful for operations where u need to quickly run over many diff values or perform certain computations that can be expressed using linear algebra. 

The most compact form of a graph, especially with many edges.  this is a key factor when graphs are huge. And it's the fastest representation if u need to check whether a particaluar pair of cities is connected.

Note: no code implemented in this part.

### Usefulness Summary

Global – Most useful if need to regularly look at all of the edges. 

Adjacency list - useful in most typical graph operations, I.e. airline connections, social networks, etc. 
Works well because most graph algorithms are already designed to work just looking at one node at a time and its neighbors.  

Adjecency matrix - Useful for operations where u need to quickly run over many diff values or perform certain computations that can be expressed using linear algebra.  The most compact form of a graph, especially with many edges.  the fastest representation if u need to check whether a particaluar pair of cities is connected.